<a href="https://colab.research.google.com/github/Braga19/ClipSoundSymb/blob/main/Classification_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 6.8 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/Thesis2023

/content/drive/MyDrive/Thesis2023


In [ ]:
from src import read, classification, adjust
from transformers import AutoFeatureExtractor, AutoModelForImageClassification

In [ ]:
!cp -r /content/Thesis2023 /content/drive/MyDrive/Thesis2023

In [ ]:
df = read.read_df('/content/drive/MyDrive/Thesis2023/dataset/avgRatings_annotated.csv')
root_dir = '/content/drive/MyDrive/Character_images_firstgen'
read.mount()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Gender Classification

In [ ]:
#subset df for names with ratings for gender(-50 male, 50 female)
gender_df = adjust.subset(df, 'name', 'rating.mean','attribute','gender')
gender_df['name'] = gender_df['name'].str.lower()

#Classifier Gender
extractor = AutoFeatureExtractor.from_pretrained("rizvandwiki/gender-classification")
model = AutoModelForImageClassification.from_pretrained("rizvandwiki/gender-classification")

#Classifcation gender
result = classification.gender(model, extractor, gender_df, root_dir)
gender_prediction = result.to_csv('gender_prediction.csv', index=False)
!cp gender_prediction.csv /content/drive/MyDrive/Thesis2023/dataset

<ipython-input-4-e8ab7569f489>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_df_5['name'] = gender_df_5['name'].str.lower()


##Trial with a subset of gender, some functions to merge predictions_df and dataset with ratings




In [ ]:
#subsetting five elements for trial
gender_df_5 = gender_df[0:1]
gender_df_5['name'] = gender_df_5['name'].str.lower()

import pandas as pd
predict_df = pd.DataFrame.from_dict(result)
predict_df.head()

,name,image_idx,prompt,female_prob,male_prob,predicted_label
0,adelaide,0,a face,0.120466,0.879534,male
1,adelaide,1,a face,0.975755,0.024245,female
2,adelaide,2,a face,0.950901,0.049099,female
3,adelaide,3,a face,0.809180,0.190820,female
4,adelaide,4,a face,0.967881,0.032119,female


In [ ]:
predict_df.groupby(['name', 'prompt'])['female_prob'].mean().reset_index()

,name,prompt,female_prob
0,adelaide,a drawing,0.973962
1,adelaide,a face,0.841519
2,adelina,a drawing,0.979558
3,adelina,a face,0.944639
4,alasdair,a drawing,0.224226
5,alasdair,a face,0.090280
6,alastor,a drawing,0.739478
7,alastor,a face,0.627404
8,alecto,a drawing,0.843258
9,alecto,a face,0.310093


In [ ]:
predict_df_avg = predict_df.groupby('name')['female_prob'].mean().reset_index()

In [ ]:
merged_df = pd.merge(predict_df_avg, gender_df_5[['name', 'rating.scale']], on="name")


In [ ]:
merged_df

,name,female_prob,rating.scale
0,adelaide,0.907740,0.957273
1,adelina,0.962099,0.977714
2,alasdair,0.157253,0.143429
3,alastor,0.683441,0.111667
4,alecto,0.576675,0.142778


# Emotion Classification

In [ ]:
#subsetting df with names with polarity ratings
emotion_df = adjust.subset(df, 'name', 'rating.mean','attribute', 'polarity')
emotion_df['name'] = emotion_df['name'].str.lower()

#Classifier emotions 
extractor1 = AutoFeatureExtractor.from_pretrained("kdhht2334/autotrain-diffusion-emotion-facial-expression-recognition-40429105176")
model1 = AutoModelForImageClassification.from_pretrained("kdhht2334/autotrain-diffusion-emotion-facial-expression-recognition-40429105176")

#Emotions classifcation
result_emotion = classification.emotion(model1, extractor1, emotion_df, root_dir)
emotion_prediction = result_emotion.to_csv('emotion_prediction.csv', index=False)
!cp emotion_prediction.csv /content/drive/MyDrive/Thesis2023/dataset


## Some functions to merge datasets

In [ ]:
import pandas as pd
#subsetting five elements for trial
emotion_df_5 = emotion_df[0:1]
emotion_df_5['name'] = emotion_df_5['name'].str.lower()
predicted_avg_emotion = result_emotion.groupby(['name', 'prompt'])['angry_prob','disgust_prob', 'fear_prob','happy_prob', 'neutral_prob', 'sad_prob'].mean().reset_index()
merged_df_emotion = pd.merge(predicted_avg_emotion, emotion_df_5[['name', 'rating.mean']], on="name")

In [ ]:
merged_df_emotion

,name,prompt,angry_prob,disgust_prob,fear_prob,happy_prob,neutral_prob,sad_prob,rating.mean
0,adelina,a drawing,0.249843,0.149661,0.048014,0.025402,0.201438,0.268475,0.816216
1,adelina,a face,0.180876,0.156538,0.115673,0.115465,0.217033,0.149888,0.816216
2,amabala,a drawing,0.184601,0.208316,0.083773,0.034076,0.171241,0.246914,0.559355
3,amabala,a face,0.258728,0.121803,0.088905,0.062573,0.221058,0.201855,0.559355
4,apple,a drawing,0.184836,0.151012,0.132519,0.048666,0.213586,0.183432,0.824444
5,apple,a face,0.177927,0.148263,0.198249,0.083212,0.182017,0.138400,0.824444
6,arcturus,a drawing,0.154139,0.360681,0.058338,0.024443,0.135448,0.201983,0.388333
7,arcturus,a face,0.135363,0.277409,0.084753,0.024637,0.190418,0.208684,0.388333
8,arobynn,a drawing,0.166236,0.339792,0.053264,0.032784,0.138485,0.202663,0.576452
9,arobynn,a face,0.132052,0.327242,0.062249,0.071896,0.071743,0.112057,0.576452


# Age Classification

In [ ]:
#subsetting names with ratings for age
age_df = adjust.subset(df, 'name', 'rating.mean','attribute', 'age')
age_df['name'] = age_df['name'].str.lower()

#classifier age
extractor_age = AutoFeatureExtractor.from_pretrained("nateraw/vit-age-classifier")
model_age = AutoModelForImageClassification.from_pretrained("nateraw/vit-age-classifier")


<ipython-input-10-5eb1fcd3054b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df_5['name'] = age_df_5['name'].str.lower()


/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
result_age = classification.age(model_age, extractor_age, age_df_5, root_dir)
age_prediction = result_age.to_csv('age_prediction.csv', index=False)
!cp age_prediction.csv /content/drive/MyDrive/Thesis2023/dataset

NameError: ignored

In [ ]:
predicted_avg_age = result_age.groupby(['name', 'prompt'])['young_prob', 'old_prob'].mean().reset_index()

<ipython-input-11-13f0fd1ec919>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  predicted_avg_age = result_age.groupby(['name', 'prompt'])['young_prob', 'old_prob'].mean().reset_index()


In [ ]:
#subsetting five elements for trial
age_df_5 = age_df[0:1]
age_df_5['name'] = age_df_5['name'].str.lower()

import pandas as pd
merged_df_age = pd.merge(predicted_avg_age, age_df_5[['name', 'rating.mean']], on="name")

In [ ]:
merged_df_age

,name,prompt,young_prob,old_prob,rating.mean
0,adelaide,a drawing,0.925272,0.074728,0.493824
1,adelaide,a face,0.965623,0.034377,0.493824
2,alasdair,a drawing,0.855728,0.144272,0.687097
3,alasdair,a face,0.970047,0.029953,0.687097
4,alastor,a drawing,0.786504,0.213496,0.638125
5,alastor,a face,0.757266,0.242734,0.638125
6,alecto,a drawing,0.744765,0.255235,0.535937
7,alecto,a face,0.875773,0.124227,0.535937
8,alice,a drawing,0.907039,0.092961,0.360303
9,alice,a face,0.874444,0.125556,0.360303
